# Advanced Retrieval Evaulation

Please read the full article at [thedataguy.pro](https://thedataguy.pro/blog/evaluating-advanced-rag-retrievers/).


## Solution Approach

### Step 1: Generate Synthetic Data

We'll generate test data using RAGAS Synthetic Data Generator to create a comprehensive evaluation set for our retrievers. The synthetic data will be stored in CSV format for reuse across different retriever evaluations.

> Note: `testset.csv` was generated using `grok-3` and `Snowflake/snowflake-arctic-embed-l` in the `grok_3.ipynb` notebook.

### Step 2: Generate Eval Dataset

We'll create a LangSmith dataset and run evaluation using the following RAGAS metrics:
- `LLMContextRecall`: Measures how much of the relevant context the LLM incorporated
- `Faithfulness`: Evaluates if the answers are grounded in the retrieved context
- `ContextRecall`: Assesses how well the retriever found relevant documents
- `AnswerRelevancy`: Measures how relevant the generated answers are to the questions

### Step 3: Evaluation

Each retriever chain will be evaluated using LangSmith experiments:
- Naive Retriever (Vector similarity)
- BM25 Retriever (Sparse retrieval)
- Contextual Compression Retriever with Cohere Rerank
- Multi-Query Retriever
- Parent Document Retriever
- Ensemble Retriever

## Step 4: Analysis and Report

We'll build comparison charts and tables across three key dimensions:
- Performance: Based on RAGAS metrics scores
- Cost: API costs for embedding, retrieval, and reranking operations
- Latency: Response time measurements for each retriever

The final report will include recommendations on which retriever performs best for this specific dataset and use case.

[Read full article](https://thedataguy.pro/blog/evaluating-advanced-rag-retrievers/)


Open `Advanced_Retrieval_Evaluation` in `VS Code` and run `uv sync`

### Required API Keys

This notebook requires the following API keys to function properly:

1. **OpenAI API Key** - For embeddings and LLM access
2. **Cohere API Key** - For reranking capability
3. **LangChain API Key** - For LangSmith tracing and evaluation
4. **X AI API Key** - For synthetic data generation

The API keys will be securely collected in the following cells using `getpass` to avoid exposing sensitive information.

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [ ]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

In [ ]:
# For LangSmith
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
dataset_name = "Retrieval Evaulation - John Wick"
os.environ["LANGCHAIN_PROJECT"] = dataset_name

In [ ]:
os.environ["XAI_API_KEY"] = getpass.getpass("Enter your XAI API Key:")

### Data Collection

We can simply `wget` these from GitHub.

In [ ]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv -O john_wick_1.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv -O john_wick_2.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv -O john_wick_3.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv -O john_wick_4.csv

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

- Self-Query: Wants as much metadata as we can provide
- Time-weighted: Wants temporal data

> NOTE: While we're creating a temporal relationship based on when these movies came out for illustrative purposes, it needs to be clear that the "time-weighting" in the Time-weighted Retriever is based on when the document was *accessed* last - not when it was created.

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

Let's look at an example document to see if everything worked as expected!

In [ ]:
documents[0]

## Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "JohnWick".

In [ ]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

## RAG Prompt Template

In [ ]:
RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""



In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser


chat_model = ChatOpenAI(model="gpt-4.1-nano")

def create_rag_chain(llm,retriever, template=RAG_TEMPLATE):
    """Create a RAG chain using the provided vectorstore and template."""
    prompt = ChatPromptTemplate.from_template(template)
    
    return (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"answer": prompt | llm | StrOutputParser(), "contexts": itemgetter("context")}
    )



## Naive RAG Chain

In [ ]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

naive_retrieval_chain = create_rag_chain(
    chat_model, retriever=naive_retriever, template=RAG_TEMPLATE
)


In [ ]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})

Overall, this is not bad! Let's see if we can make it better!

## Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [ ]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

bm25_retrieval_chain = create_rag_chain(
    chat_model, retriever=bm25_retriever, template=RAG_TEMPLATE
)

Let's look at the responses!

In [ ]:
bm25_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["answer"]

It's not clear that this is better or worse - but the `I don't know` isn't great!

## Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [ ]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

contextual_compression_retrieval_chain = create_rag_chain(
    chat_model, retriever=compression_retriever, template=RAG_TEMPLATE
)

Let's look at the responses!

In [ ]:
contextual_compression_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["answer"]

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

multi_query_retrieval_chain = create_rag_chain(
    chat_model, retriever=multi_query_retriever, template=RAG_TEMPLATE
)

In [ ]:
multi_query_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["answer"]

## Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [ ]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)
parent_document_retriever.add_documents(parent_docs, ids=None)

parent_document_retrieval_chain = create_rag_chain(
    chat_model, retriever=parent_document_retriever, template=RAG_TEMPLATE
)

Let's give it a whirl!

In [ ]:
parent_document_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["answer"]

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [ ]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

ensemble_retrieval_chain = create_rag_chain(
    chat_model, retriever=ensemble_retriever, template=RAG_TEMPLATE
)

Let's look at our results!

In [ ]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["answer"]

## Generate Synthetic Data

In [ ]:
from ragas.run_config import RunConfig
from ragas.testset import TestsetGenerator
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_xai import ChatXAI

grok = ChatXAI(model="grok-3-latest")
# Create a RunConfig with rate limit settings
my_run_config = RunConfig(
    max_workers=8,      # Control concurrent requests (default is 16)
    timeout=180,         # Maximum time to wait for a single operation (default is 180s)
    max_retries=10,      # Maximum number of retry attempts (default is 10)
    max_wait=120,         # Maximum wait time between retries (default is 60s)
    exception_types=(Exception,)  # Types of exceptions to retry on
)

# Initialize the generator with the LLM and embedding model
generator_llm = LangchainLLMWrapper(grok)
generator_embeddings = LangchainEmbeddingsWrapper(HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-l"))

# Set the run config for both LLM and embeddings
generator_llm.set_run_config(my_run_config)
generator_embeddings.set_run_config(my_run_config)

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)

# Use the run_config in your generate call
dataset = generator.generate_with_langchain_docs(
    documents=documents, 
    testset_size=10,
    run_config=my_run_config
)
# Save the testset to a CSV file
dataset.to_pandas().to_csv("testset.csv", index=False)

## Upload LangSmith DataSet

In [ ]:
# from ragas.integrations.langsmith import upload_dataset

from __future__ import annotations
import pandas as pd
import ast

import typing as t

from langchain.smith import RunEvalConfig

from ragas.integrations.langchain import EvaluatorChain

if t.TYPE_CHECKING:
    from langsmith.schemas import Dataset as LangsmithDataset

    from ragas.testset import Testset

try:
    from langsmith import Client
    from langsmith.utils import LangSmithNotFoundError
except ImportError:
    raise ImportError(
        "Please install langsmith to use this feature. You can install it via pip install langsmith"
    )


def upload_dataset(
    dataset: pd.DataFrame, dataset_name: str, dataset_desc: str = ""
) -> LangsmithDataset:
    """
    Uploads a new dataset to LangSmith, converting it from a TestDataset object to a
    pandas DataFrame before upload. If a dataset with the specified name already
    exists, the function raises an error.

    Parameters
    ----------
    dataset : TestDataset
        The dataset to be uploaded.
    dataset_name : str
        The name for the new dataset in LangSmith.
    dataset_desc : str, optional
        A description for the new dataset. The default is an empty string.

    Returns
    -------
    LangsmithDataset
        The dataset object as stored in LangSmith after upload.

    Raises
    ------
    ValueError
        If a dataset with the specified name already exists in LangSmith.

    Notes
    -----
    The function attempts to read a dataset by the given name to check its existence.
    If not found, it proceeds to upload the dataset after converting it to a pandas
    DataFrame. This involves specifying input and output keys for the dataset being
    uploaded.
    """
    client = Client()
    try:
        # check if dataset exists
        langsmith_dataset: LangsmithDataset = client.read_dataset(
            dataset_name=dataset_name
        )
        raise ValueError(
            f"Dataset {dataset_name} already exists in langsmith. [{langsmith_dataset}]"
        )
    except LangSmithNotFoundError:
        # if not create a new one with the generated query examples
        langsmith_dataset: LangsmithDataset = client.upload_dataframe(
            df=dataset,
            name=dataset_name,
            input_keys=["question"],
            output_keys=["ground_truth"],
            #metadata_keys=["context"],
            description=dataset_desc,
        )

        print(
            f"Created a new dataset '{langsmith_dataset.name}'. Dataset is accessible at {langsmith_dataset.url}"
        )
        return langsmith_dataset
    
# Load the test set from a CSV file

df = pd.read_csv("testset.csv")
# Convert string representations of lists to actual Python lists
df['reference_contexts'] = df['reference_contexts'].apply(ast.literal_eval)
# set columns to question, context, ground_truth
df = df.rename(columns={
    'user_input': 'question',
    'reference_contexts': 'context',
    'reference': 'ground_truth'
})

upload_dataset(
    dataset=df,
    dataset_name=dataset_name,
    dataset_desc="A test set of John Wick reviews",
)

## Run Evaluation

In [ ]:
from ragas.integrations.langsmith import evaluate
from ragas.metrics import context_recall, faithfulness, context_precision, answer_relevancy

# build the evaluation metrics
metrics = [answer_relevancy, context_precision, faithfulness, context_recall]

# Create a list of chains to evaluate
chain_list = [
    ("Naive Retrieval", naive_retrieval_chain),
    ("BM25 Retrieval", bm25_retrieval_chain),
    ("Parent Document Retrieval", parent_document_retrieval_chain),
    ("Contextual Compression Retrieval", contextual_compression_retrieval_chain),
    ("Multi-Query Retrieval", multi_query_retrieval_chain),
    ("Ensemble Retrieval", ensemble_retrieval_chain),
]


# Run evaluation on each chain
for chain_name, chain in chain_list:
    print(f"Evaluating {chain_name}...")

    evaluate(
        dataset_name=dataset_name,
        llm_or_chain_factory=chain,
        experiment_name=f"{chain_name}",
        metrics=metrics,
    )
